# 个贷违约预测

## 1. 数据预处理

### 1.1 导入数据

In [2]:
import numpy as np
import pandas as pd
#设置不隐藏列
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data = pd.read_csv('./data.csv')

### 1.2 查看数据

In [3]:
#查看前5行
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault
0,1040418,240418,31818.18182,3,11.466,1174.91,C,政府机构,金融业,3 years,0,2016/10/1,2,2.430000,0,3,0,0.0,3,9927,0.0,0
1,1025197,225197,28000.00000,5,16.841,670.69,C,政府机构,金融业,10+ years,0,2013/6/1,0,11.004545,1,3,0,0.0,0,0,0.0,0
2,1009360,209360,17272.72727,3,8.900,603.32,A,政府机构,公共服务、社会组织,10+ years,1,2014/1/1,4,6.409091,0,5,0,0.0,0,0,0.0,0
3,1039708,239708,20000.00000,3,4.788,602.30,A,世界五百强,文化和体育业,6 years,0,2015/7/1,0,9.204545,0,3,0,0.0,0,0,0.0,0
4,1027483,227483,15272.72727,3,12.790,470.31,C,政府机构,信息传输、软件和信息技术服务业,< 1 year,2,2016/7/1,0,15.578182,0,15,0,0.0,0,0,0.0,0


In [4]:
#数据维度
data.shape

(10000, 22)

In [5]:
#数据整体情况
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   loan_id                   10000 non-null  int64  
 1   user_id                   10000 non-null  int64  
 2   total_loan                10000 non-null  float64
 3   year_of_loan              10000 non-null  int64  
 4   interest                  10000 non-null  float64
 5   monthly_payment           10000 non-null  float64
 6   class                     10000 non-null  object 
 7   employer_type             10000 non-null  object 
 8   industry                  10000 non-null  object 
 9   work_year                 9378 non-null   object 
 10  house_exist               10000 non-null  int64  
 11  issue_date                10000 non-null  object 
 12  use                       10000 non-null  int64  
 13  debt_loan_ratio           10000 non-null  float64
 14  del_in_

In [6]:
#查看数据统计信息
data.describe()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,house_exist,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9993.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.025210e+06,225209.587700,14402.126591,3.479600,13.222782,436.960427,0.612200,1.762600,17.532170,0.311600,11.645000,0.226400,0.138997,1.290600,2173.916100,335.232240,0.168300
std,1.438682e+04,14386.820956,8953.946807,0.853965,4.875755,261.754396,0.672203,2.392031,14.218509,0.872457,5.500745,0.607931,0.379089,1.449469,3027.539977,635.108825,0.374151
min,1.000008e+06,200008.000000,818.181818,3.000000,4.779000,30.440000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.012973e+06,212973.250000,7500.000000,3.000000,9.702000,248.820000,0.000000,0.000000,11.157955,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.025276e+06,225276.500000,12272.727270,3.000000,12.639000,371.525000,1.000000,0.000000,16.652273,0.000000,11.000000,0.000000,0.000000,1.000000,838.500000,0.000000,0.000000
75%,1.037694e+06,237694.500000,19636.363640,3.000000,15.985500,573.830000,1.000000,4.000000,22.781591,0.000000,14.000000,0.000000,0.000000,3.000000,3354.250000,413.675000,0.000000
max,1.049997e+06,249997.000000,47272.727270,5.000000,33.979000,1503.890000,4.000000,13.000000,999.000000,15.000000,59.000000,12.000000,5.000000,5.000000,18413.000000,5523.900000,1.000000


### 1.3 检查数据质量

In [7]:
#查看数据缺失情况
data.isnull().sum()

loan_id                       0
user_id                       0
total_loan                    0
year_of_loan                  0
interest                      0
monthly_payment               0
class                         0
employer_type                 0
industry                      0
work_year                   622
house_exist                   0
issue_date                    0
use                           0
debt_loan_ratio               0
del_in_18month                0
known_outstanding_loan        0
known_dero                    0
pub_dero_bankrup              7
early_return                  0
early_return_amount           0
early_return_amount_3mon      0
isDefault                     0
dtype: int64

work_year与pub_dero_bankrup数据存在缺失，尝试缺失值补全。

### 1.4 离散属性编码

#### 需要编码的离散属性为class,employer_type,industry,work_year,issue_date

1.对class进行编码

In [8]:
#作出class与isDefault的列联表
class_anl = pd.crosstab(data['class'], data['isDefault'])
class_anl['ratio'] = class_anl.apply(lambda row: row[1] / row[0], axis=1)
print(class_anl.columns)
print(class_anl)

Index([0, 1, 'ratio'], dtype='object', name='isDefault')
isDefault     0    1     ratio
class                         
A          1597  101  0.063244
B          2617  348  0.132977
C          2303  556  0.241424
D          1139  344  0.302019
E           468  225  0.480769
F           155   86  0.554839
G            38   23  0.605263


In [9]:
#利用OrdinalEncoder对class进行序数编码
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories = [['A','B','C','D','E','F','G']])
data['class'] = encoder.fit_transform(data[['class']])
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault
0,1040418,240418,31818.18182,3,11.466,1174.91,2.0,政府机构,金融业,3 years,0,2016/10/1,2,2.430000,0,3,0,0.0,3,9927,0.0,0
1,1025197,225197,28000.00000,5,16.841,670.69,2.0,政府机构,金融业,10+ years,0,2013/6/1,0,11.004545,1,3,0,0.0,0,0,0.0,0
2,1009360,209360,17272.72727,3,8.900,603.32,0.0,政府机构,公共服务、社会组织,10+ years,1,2014/1/1,4,6.409091,0,5,0,0.0,0,0,0.0,0
3,1039708,239708,20000.00000,3,4.788,602.30,0.0,世界五百强,文化和体育业,6 years,0,2015/7/1,0,9.204545,0,3,0,0.0,0,0,0.0,0
4,1027483,227483,15272.72727,3,12.790,470.31,2.0,政府机构,信息传输、软件和信息技术服务业,< 1 year,2,2016/7/1,0,15.578182,0,15,0,0.0,0,0,0.0,0


2.对employer_type进行编码

In [10]:
data = data.join(pd.get_dummies(data['employer_type'], prefix ='employer_tpye', drop_first = True))
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,employer_tpye_世界五百强,employer_tpye_幼教与中小学校,employer_tpye_政府机构,employer_tpye_普通企业,employer_tpye_高等教育机构
0,1040418,240418,31818.18182,3,11.466,1174.91,2.0,政府机构,金融业,3 years,0,2016/10/1,2,2.430000,0,3,0,0.0,3,9927,0.0,0,False,False,True,False,False
1,1025197,225197,28000.00000,5,16.841,670.69,2.0,政府机构,金融业,10+ years,0,2013/6/1,0,11.004545,1,3,0,0.0,0,0,0.0,0,False,False,True,False,False
2,1009360,209360,17272.72727,3,8.900,603.32,0.0,政府机构,公共服务、社会组织,10+ years,1,2014/1/1,4,6.409091,0,5,0,0.0,0,0,0.0,0,False,False,True,False,False
3,1039708,239708,20000.00000,3,4.788,602.30,0.0,世界五百强,文化和体育业,6 years,0,2015/7/1,0,9.204545,0,3,0,0.0,0,0,0.0,0,True,False,False,False,False
4,1027483,227483,15272.72727,3,12.790,470.31,2.0,政府机构,信息传输、软件和信息技术服务业,< 1 year,2,2016/7/1,0,15.578182,0,15,0,0.0,0,0,0.0,0,False,False,True,False,False


3.对industry进行编码

In [11]:
data = data.join(pd.get_dummies(data['industry'], prefix ='industry', drop_first = True))
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,employer_tpye_世界五百强,employer_tpye_幼教与中小学校,employer_tpye_政府机构,employer_tpye_普通企业,employer_tpye_高等教育机构,industry_住宿和餐饮业,industry_信息传输、软件和信息技术服务业,industry_公共服务、社会组织,industry_农、林、牧、渔业,industry_制造业,industry_国际组织,industry_建筑业,industry_房地产业,industry_批发和零售业,industry_文化和体育业,industry_电力、热力生产供应业,industry_采矿业,industry_金融业
0,1040418,240418,31818.18182,3,11.466,1174.91,2.0,政府机构,金融业,3 years,0,2016/10/1,2,2.430000,0,3,0,0.0,3,9927,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,1025197,225197,28000.00000,5,16.841,670.69,2.0,政府机构,金融业,10+ years,0,2013/6/1,0,11.004545,1,3,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,1009360,209360,17272.72727,3,8.900,603.32,0.0,政府机构,公共服务、社会组织,10+ years,1,2014/1/1,4,6.409091,0,5,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,1039708,239708,20000.00000,3,4.788,602.30,0.0,世界五百强,文化和体育业,6 years,0,2015/7/1,0,9.204545,0,3,0,0.0,0,0,0.0,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4,1027483,227483,15272.72727,3,12.790,470.31,2.0,政府机构,信息传输、软件和信息技术服务业,< 1 year,2,2016/7/1,0,15.578182,0,15,0,0.0,0,0,0.0,0,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


4.对work_year进行编码

In [12]:
def convert_work(work_year):
    work_year=str(work_year).strip()
    # < 1 year 转为 0
    if work_year == '< 1 year':
        return 0
    # 10+ years 转为 11
    elif work_year == '10+ years':
        return 11
    # 其余年份提取年份数值
    else:
        try:
            return int(work_year.split()[0])
        except (ValueError):
            return np.nan
data['work_year'] = data['work_year'].apply(convert_work)
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,employer_tpye_世界五百强,employer_tpye_幼教与中小学校,employer_tpye_政府机构,employer_tpye_普通企业,employer_tpye_高等教育机构,industry_住宿和餐饮业,industry_信息传输、软件和信息技术服务业,industry_公共服务、社会组织,industry_农、林、牧、渔业,industry_制造业,industry_国际组织,industry_建筑业,industry_房地产业,industry_批发和零售业,industry_文化和体育业,industry_电力、热力生产供应业,industry_采矿业,industry_金融业
0,1040418,240418,31818.18182,3,11.466,1174.91,2.0,政府机构,金融业,3.0,0,2016/10/1,2,2.430000,0,3,0,0.0,3,9927,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,1025197,225197,28000.00000,5,16.841,670.69,2.0,政府机构,金融业,11.0,0,2013/6/1,0,11.004545,1,3,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,1009360,209360,17272.72727,3,8.900,603.32,0.0,政府机构,公共服务、社会组织,11.0,1,2014/1/1,4,6.409091,0,5,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,1039708,239708,20000.00000,3,4.788,602.30,0.0,世界五百强,文化和体育业,6.0,0,2015/7/1,0,9.204545,0,3,0,0.0,0,0,0.0,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4,1027483,227483,15272.72727,3,12.790,470.31,2.0,政府机构,信息传输、软件和信息技术服务业,0.0,2,2016/7/1,0,15.578182,0,15,0,0.0,0,0,0.0,0,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


5.对issue_date进行编码

In [13]:
data['issue_date'] = pd.to_datetime(data['issue_date'])
data['issue_year'] = data['issue_date'].dt.year
data['issue_month'] = data['issue_date'].dt.month
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,employer_tpye_世界五百强,employer_tpye_幼教与中小学校,employer_tpye_政府机构,employer_tpye_普通企业,employer_tpye_高等教育机构,industry_住宿和餐饮业,industry_信息传输、软件和信息技术服务业,industry_公共服务、社会组织,industry_农、林、牧、渔业,industry_制造业,industry_国际组织,industry_建筑业,industry_房地产业,industry_批发和零售业,industry_文化和体育业,industry_电力、热力生产供应业,industry_采矿业,industry_金融业,issue_year,issue_month
0,1040418,240418,31818.18182,3,11.466,1174.91,2.0,政府机构,金融业,3.0,0,2016-10-01,2,2.430000,0,3,0,0.0,3,9927,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,2016,10
1,1025197,225197,28000.00000,5,16.841,670.69,2.0,政府机构,金融业,11.0,0,2013-06-01,0,11.004545,1,3,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,2013,6
2,1009360,209360,17272.72727,3,8.900,603.32,0.0,政府机构,公共服务、社会组织,11.0,1,2014-01-01,4,6.409091,0,5,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,2014,1
3,1039708,239708,20000.00000,3,4.788,602.30,0.0,世界五百强,文化和体育业,6.0,0,2015-07-01,0,9.204545,0,3,0,0.0,0,0,0.0,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,2015,7
4,1027483,227483,15272.72727,3,12.790,470.31,2.0,政府机构,信息传输、软件和信息技术服务业,0.0,2,2016-07-01,0,15.578182,0,15,0,0.0,0,0,0.0,0,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,2016,7


6. 删除无用变量

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   loan_id                   10000 non-null  int64         
 1   user_id                   10000 non-null  int64         
 2   total_loan                10000 non-null  float64       
 3   year_of_loan              10000 non-null  int64         
 4   interest                  10000 non-null  float64       
 5   monthly_payment           10000 non-null  float64       
 6   class                     10000 non-null  float64       
 7   employer_type             10000 non-null  object        
 8   industry                  10000 non-null  object        
 9   work_year                 9378 non-null   float64       
 10  house_exist               10000 non-null  int64         
 11  issue_date                10000 non-null  datetime64[ns]
 12  use                

In [15]:
data = data.drop(columns=['employer_type','industry','issue_date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   loan_id                   10000 non-null  int64  
 1   user_id                   10000 non-null  int64  
 2   total_loan                10000 non-null  float64
 3   year_of_loan              10000 non-null  int64  
 4   interest                  10000 non-null  float64
 5   monthly_payment           10000 non-null  float64
 6   class                     10000 non-null  float64
 7   work_year                 9378 non-null   float64
 8   house_exist               10000 non-null  int64  
 9   use                       10000 non-null  int64  
 10  debt_loan_ratio           10000 non-null  float64
 11  del_in_18month            10000 non-null  int64  
 12  known_outstanding_loan    10000 non-null  int64  
 13  known_dero                10000 non-null  int64  
 14  pub_der

### 1.5 数据清洗

1. 用**众数**填补**pub_dero_bankrup**缺失值

In [16]:
data['pub_dero_bankrup'].value_counts()

pub_dero_bankrup
0.0    8697
1.0    1223
2.0      58
3.0      12
5.0       2
4.0       1
Name: count, dtype: int64

In [17]:
data['pub_dero_bankrup'] = data['pub_dero_bankrup'].fillna(0)
data.isnull().sum()

loan_id                       0
user_id                       0
total_loan                    0
year_of_loan                  0
interest                      0
monthly_payment               0
class                         0
work_year                   622
house_exist                   0
use                           0
debt_loan_ratio               0
del_in_18month                0
known_outstanding_loan        0
known_dero                    0
pub_dero_bankrup              0
early_return                  0
early_return_amount           0
early_return_amount_3mon      0
isDefault                     0
employer_tpye_世界五百强           0
employer_tpye_幼教与中小学校         0
employer_tpye_政府机构            0
employer_tpye_普通企业            0
employer_tpye_高等教育机构          0
industry_住宿和餐饮业               0
industry_信息传输、软件和信息技术服务业      0
industry_公共服务、社会组织            0
industry_农、林、牧、渔业             0
industry_制造业                  0
industry_国际组织                 0
industry_建筑业                  0
industry

2. 用**KNN法**填补**work_year**缺失值

In [18]:
from sklearn.impute import KNNImputer
#选取数值变量
data_numeric = data[['total_loan','interest','monthly_payment',
                     'class','work_year','house_exist','use',
                     'debt_loan_ratio','del_in_18month','known_outstanding_loan',
                     'known_dero','pub_dero_bankrup','early_return',
                     'early_return_amount','early_return_amount_3mon','issue_year',
                     'issue_month']].to_numpy()
imputer = KNNImputer(n_neighbors=5, weights='uniform')
data_numeric_impute = imputer.fit_transform(data_numeric)
column_name = ['total_loan','interest','monthly_payment',
                     'class','work_year','house_exist','use',
                     'debt_loan_ratio','del_in_18month','known_outstanding_loan',
                     'known_dero','pub_dero_bankrup','early_return',
                     'early_return_amount','early_return_amount_3mon','issue_year',
                     'issue_month']
data_numeric_impute_df = pd.DataFrame(data_numeric_impute, columns=column_name)
data['work_year'] = data_numeric_impute_df['work_year']
data.isnull().sum()

loan_id                     0
user_id                     0
total_loan                  0
year_of_loan                0
interest                    0
monthly_payment             0
class                       0
work_year                   0
house_exist                 0
use                         0
debt_loan_ratio             0
del_in_18month              0
known_outstanding_loan      0
known_dero                  0
pub_dero_bankrup            0
early_return                0
early_return_amount         0
early_return_amount_3mon    0
isDefault                   0
employer_tpye_世界五百强         0
employer_tpye_幼教与中小学校       0
employer_tpye_政府机构          0
employer_tpye_普通企业          0
employer_tpye_高等教育机构        0
industry_住宿和餐饮业             0
industry_信息传输、软件和信息技术服务业    0
industry_公共服务、社会组织          0
industry_农、林、牧、渔业           0
industry_制造业                0
industry_国际组织               0
industry_建筑业                0
industry_房地产业               0
industry_批发和零售业             0
industry_文

### 1.6 数据标准化

In [19]:
#选取range不在[0,1]的变量
data.describe()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,work_year,house_exist,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,issue_year,issue_month
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.025210e+06,225209.587700,14402.126591,3.479600,13.222782,436.960427,1.747500,6.415800,0.612200,1.762600,17.532170,0.311600,11.645000,0.226400,0.138900,1.290600,2173.916100,335.232240,0.168300,2014.959600,6.512200
std,1.438682e+04,14386.820956,8953.946807,0.853965,4.875755,261.754396,1.285344,3.978359,0.672203,2.392031,14.218509,0.872457,5.500745,0.607931,0.378974,1.449469,3027.539977,635.108825,0.374151,1.634391,3.473767
min,1.000008e+06,200008.000000,818.181818,3.000000,4.779000,30.440000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2007.000000,1.000000
25%,1.012973e+06,212973.250000,7500.000000,3.000000,9.702000,248.820000,1.000000,3.000000,0.000000,0.000000,11.157955,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2014.000000,3.000000
50%,1.025276e+06,225276.500000,12272.727270,3.000000,12.639000,371.525000,2.000000,6.000000,1.000000,0.000000,16.652273,0.000000,11.000000,0.000000,0.000000,1.000000,838.500000,0.000000,0.000000,2015.000000,7.000000
75%,1.037694e+06,237694.500000,19636.363640,3.000000,15.985500,573.830000,2.000000,11.000000,1.000000,4.000000,22.781591,0.000000,14.000000,0.000000,0.000000,3.000000,3354.250000,413.675000,0.000000,2016.000000,10.000000
max,1.049997e+06,249997.000000,47272.727270,5.000000,33.979000,1503.890000,6.000000,11.000000,4.000000,13.000000,999.000000,15.000000,59.000000,12.000000,5.000000,5.000000,18413.000000,5523.900000,1.000000,2018.000000,12.000000


In [20]:
data_r = data[['total_loan','year_of_loan','interest','monthly_payment',
               'class','work_year','house_exist','use','debt_loan_ratio',
               'del_in_18month','known_outstanding_loan','known_dero',
               'pub_dero_bankrup','early_return','early_return_amount',
               'early_return_amount_3mon','issue_year','issue_month']]
#正态标准化
from sklearn import preprocessing
z_scaler = preprocessing.StandardScaler().fit(data_r)
data[['total_loan','year_of_loan','interest','monthly_payment',
               'class','work_year','house_exist','use','debt_loan_ratio',
               'del_in_18month','known_outstanding_loan','known_dero',
               'pub_dero_bankrup','early_return','early_return_amount',
               'early_return_amount_3mon','issue_year','issue_month']] = z_scaler.transform(data_r)
data.describe()


,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,work_year,house_exist,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,issue_year,issue_month
count,1.000000e+04,10000.000000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,1.000000e+04,1.000000e+04
mean,1.025210e+06,225209.587700,2.351896e-16,-2.025047e-17,1.373124e-16,6.750156e-17,-4.103384e-17,-1.421085e-18,3.552714e-17,3.694822e-17,1.847411e-17,3.694822e-17,8.437695e-17,6.554757e-17,-4.263256e-18,3.055334e-17,1.136868e-17,-5.684342e-17,0.168300,5.542873e-14,-2.415845e-17
std,1.438682e+04,14386.820956,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,0.374151,1.000050e+00,1.000050e+00
min,1.000008e+06,200008.000000,-1.517166e+00,-5.616434e-01,-1.731876e+00,-1.553138e+00,-1.359626e+00,-1.612756e+00,-9.107829e-01,-7.369001e-01,-1.233114e+00,-3.571701e-01,-1.935289e+00,-3.724294e-01,-3.665344e-01,-8.904393e-01,-7.180829e-01,-5.278607e-01,0.000000,-4.870313e+00,-1.586887e+00
25%,1.012973e+06,212973.250000,-7.708859e-01,-5.616434e-01,-7.221359e-01,-7.188030e-01,-5.815855e-01,-8.586381e-01,-9.107829e-01,-7.369001e-01,-4.483265e-01,-3.571701e-01,-6.626706e-01,-3.724294e-01,-3.665344e-01,-8.904393e-01,-7.180829e-01,-5.278607e-01,0.000000,-5.871593e-01,-1.011114e+00
50%,1.025276e+06,225276.500000,-2.378287e-01,-5.616434e-01,-1.197376e-01,-2.500004e-01,1.964553e-01,-1.045207e-01,5.769382e-01,-7.369001e-01,-6.188701e-02,-3.571701e-01,-1.172627e-01,-3.724294e-01,-3.665344e-01,-2.004972e-01,-4.411116e-01,-5.278607e-01,0.000000,2.471992e-02,1.404309e-01
75%,1.037694e+06,237694.500000,5.846024e-01,-5.616434e-01,5.666520e-01,5.229193e-01,1.964553e-01,1.152342e+00,5.769382e-01,9.354025e-01,3.692148e-01,-3.571701e-01,4.281452e-01,-3.724294e-01,-3.665344e-01,1.179387e+00,3.898852e-01,1.235169e-01,0.000000,6.365991e-01,1.004090e+00
max,1.049997e+06,249997.000000,3.671257e+00,1.780489e+00,4.257239e+00,4.076275e+00,3.308619e+00,1.152342e+00,5.040102e+00,4.698083e+00,6.903093e+01,1.683651e+01,8.609264e+00,1.936764e+01,1.282765e+01,2.559271e+00,5.364057e+00,8.170139e+00,1.000000,1.860357e+00,1.579863e+00


### 1.7 特征选取

In [21]:
#删除loan_id,loan_id
data = data.drop(columns=['loan_id','user_id'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   total_loan                10000 non-null  float64
 1   year_of_loan              10000 non-null  float64
 2   interest                  10000 non-null  float64
 3   monthly_payment           10000 non-null  float64
 4   class                     10000 non-null  float64
 5   work_year                 10000 non-null  float64
 6   house_exist               10000 non-null  float64
 7   use                       10000 non-null  float64
 8   debt_loan_ratio           10000 non-null  float64
 9   del_in_18month            10000 non-null  float64
 10  known_outstanding_loan    10000 non-null  float64
 11  known_dero                10000 non-null  float64
 12  pub_dero_bankrup          10000 non-null  float64
 13  early_return              10000 non-null  float64
 14  early_r

## 2. 违约预测

### 2.1 数据集划分———训练集+验证集（交叉验证）、测试集

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
#划分X与Y
X = data.drop(['isDefault'], axis = 1)
y = data['isDefault']
#划分训练集+验证集与测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

### 2.2 逻辑回归

1. 模型构建及参数选择

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
#初始化LogisticRegression
logreg = LogisticRegression()

#定义参数网络
param_logreg = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs', 'saga'],
    'penalty': ['l2','l1'],
    'max_iter': [100, 200, 500]
}

# 使用GridSearchCV进行网格搜索，在训练集中采用5折交叉盐酸
grid_logreg = GridSearchCV(estimator=logreg, param_grid=param_logreg, cv=5, scoring='roc_auc')

# 在训练集上拟合
grid_logreg.fit(X_train, y_train)

# 输出最佳参数
print("Best Parameters:", grid_logreg.best_params_)

e:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
e:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
e:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
e:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
e:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
e:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
e:\a

Best Parameters: {'C': 0.1, 'max_iter': 500, 'penalty': 'l1', 'solver': 'saga'}


2. 效果评价

In [24]:
#预测及效果评价
print('逻辑回归得分')
logreg_best = grid_logreg.best_estimator_
y_hat_test = logreg_best.predict(X_test)
print('Precision:{:.3f}'.format(precision_score(y_test,y_hat_test)))
print('Recall:{:.3f}'.format(recall_score(y_test,y_hat_test)))
print('F1:{:.3f}'.format(f1_score(y_test,y_hat_test)))
print('AOC:{:.3f}'.format(roc_auc_score(y_test,y_hat_test)))

逻辑回归得分
Precision:0.576
Recall:0.239
F1:0.338
AOC:0.602


### 2.3 朴素贝叶斯

1. 模型构建及参数选择

In [27]:
from sklearn.naive_bayes import BernoulliNB
#初始化BernoulliNB
clf = BernoulliNB()

#定义参数网络
param_clf = {
    'alpha': [0.1, 0.5, 1, 5, 10],
    'binarize': [0, 0.5, 1]
}
grid_clf = GridSearchCV(estimator=clf, param_grid=param_clf,scoring='roc_auc')
grid_clf.fit(X_train,y_train)

print("Best Parameters:", grid_clf.best_params_)

Best Parameters: {'alpha': 0.1, 'binarize': 0}


2. 效果评价

In [28]:
#预测及效果评价
print('朴素贝叶斯得分')
clf_best = grid_clf.best_estimator_
y_hat_test = clf_best.predict(X_test)
print('Precision:{:.3f}'.format(precision_score(y_test,y_hat_test)))
print('Recall:{:.3f}'.format(recall_score(y_test,y_hat_test)))
print('F1:{:.3f}'.format(f1_score(y_test,y_hat_test)))
print('AOC:{:.3f}'.format(roc_auc_score(y_test,y_hat_test)))

朴素贝叶斯得分
Precision:0.418
Recall:0.579
F1:0.486
AOC:0.708


### 2.4 决策树

1. 模型构建及参数选择

In [36]:
from sklearn.tree import DecisionTreeClassifier
#初始化DecisionTreeClassifier
dt = DecisionTreeClassifier()

#定义网格参数
param_dt = {
    'max_depth': [3,4,5,6],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
}

grid_dt = GridSearchCV(estimator=dt, param_grid=param_dt, scoring='roc_auc')
grid_dt.fit(X_train,y_train)

print("Best Parameters:",grid_dt.best_params_)

Best Parameters: {'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 20}


2. 效果评价

In [37]:
#预测及效果评价
print('决策树得分')
dt_best = grid_dt.best_estimator_
y_hat_test = dt_best.predict(X_test)
print('Precision:{:.3f}'.format(precision_score(y_test,y_hat_test)))
print('Recall:{:.3f}'.format(recall_score(y_test,y_hat_test)))
print('F1:{:.3f}'.format(f1_score(y_test,y_hat_test)))
print('AOC:{:.3f}'.format(roc_auc_score(y_test,y_hat_test)))

决策树得分
Precision:0.561
Recall:0.373
F1:0.448
AOC:0.657
